In [1]:
import time
from selenium import webdriver
import requests
import json
import pprint
import pandas as pd

In [ ]:
#使用selenium打开爬取网页
driver = webdriver.Edge()
driver.get(r"https://www.bilibili.com/video/BV1YA4y1R7RJ?spm_id_from=3&vd_source=75c5cc72360e88d57d6f3a56dfb962cf")

#获取页码初始高度
js = "return action = document.body.scrollHeight"
height = driver.execute_script(js)

#将滚动条调整至页面底部
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(10)

#定义初始时间戳(s)
t1 = int(time.time())
#定义循环标识，用于终止while循环
status = True
#重试次数
num = 0
while status:
    #获取当前时间戳
    t2 = int(time.time())
    #判断时间初始时间戳和当前时间戳相差是否大于30s，小于30s则下拉滚动条
    if t2-t1 < 30:
        new_height = driver.execute_script(js)
        if new_height > height :
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            #重置初始页面高度
            height = new_height
            #重置初始时间戳，重新计时
            t1 = int(time.time())
    elif num < 3:
        time.sleep(5)
        num = num+1
    else:
        print("已获得足够数量评论")
        status = False
        #滚动条调整至页面顶部
        driver.execute_script("window.scrollTo(0,0)")
        break

In [ ]:
#爬取评论点赞量
data2 = []
for id in range(1,320):
    url = 'https://api.bilibili.com/x/v2/reply/main?mode=3&next={}&oid=897395126&plat=1&type=1'.format(id)
    rq = requests.get(url)
    shuju = json.loads(rq.text)    
    for i in shuju['data']['replies']:
        data2.append([i['like']]) 
        if i['replies']!= None: 
            for j in i['replies']:
                data2.append([j['like']]) 
                time.sleep(2)

#爬取置顶评论点赞量
data1 = []
for i in shuju['data']['top_replies']:
    data1.append([i['like']]) 
    if i['replies']!= None:
        for j in i['replies']:
            data1.append([j['like']])

print("爬取完成")


#整理数据
data = data1 + data2
content = pd.DataFrame(columns=['评论点赞量'],data=data)
content.to_csv('like1.csv', index=0)

In [ ]:
#爬取网页评论及子评论
data2 = []
for id in range(1,320):
    url = 'https://api.bilibili.com/x/v2/reply/main?mode=3&next={}&oid=897395126&plat=1&type=1'.format(id)
    rq = requests.get(url)
    shuju = json.loads(rq.text)    
    for i in shuju['data']['replies']:
        data2.append([i['content']['message']]) 
        if i['replies']!= None:            
            for j in i['replies']:
                data2.append([j['content']['message']])
                time.sleep(2)

#爬取置顶评论
data1 = []
for i in shuju['data']['top_replies']:
    data1.append([i['content']['message']]) 
    if i['replies']!= None:
        for j in i['replies']:
            data1.append([j['content']['message']])

print("爬取完成")
           
#整理数据
data = data1 + data2
content = pd.DataFrame(columns=['视频评论'],data=data)
content.to_csv('replys1.csv', index=0)

In [ ]:
#爬取评论发表时间
data2 = []
for id in range(1,320):
    url = 'https://api.bilibili.com/x/v2/reply/main?mode=3&next={}&oid=897395126&plat=1&type=1'.format(id)
    rq = requests.get(url)
    shuju = json.loads(rq.text)    
    for i in shuju['data']['replies']:
        data2.append([i['ctime']]) 
        if i['replies']!= None: 
            for j in i['replies']:
                data2.append([j['ctime']]) 
                time.sleep(2)

#爬取置顶评论发表时间
data1 = []
for i in shuju['data']['top_replies']:
    data1.append([i['ctime']]) 
    if i['replies']!= None:
        for j in i['replies']:
            data1.append([j['ctime']])
            
print("爬取完成")

#整理数据
data = data1 + data2
content = pd.DataFrame(columns=['评论发表时间'],data=data)
content.to_csv('ctime1.csv', index=0)

In [120]:
#整合全部csv文件
df_replys = pd.read_csv('replys1.csv',encoding='utf-8')
df_ctime = pd.read_csv('ctime1.csv',encoding='utf-8')
df_like = pd.read_csv('like1.csv',encoding='utf-8')
df = pd.concat([df_ctime , df_like, df_replys], axis=1)
df.to_csv('df.csv', index=0)